In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
from google.colab import userdata
#userdata.get('secretName')
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LENGSMITH_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = userdata.get('LANGSMITH_PROJECT')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['LANGCHAIN_ENDPOINT'] = userdata.get('LANGSMITH_ENDPOINT')
os.environ['LANGCHAIN_TRACING'] = userdata.get('LANGSMITH_TRACING')

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter


In [ ]:
import os

os.getcwd()

In [ ]:
# %cd /content/drive/MyDrive/디스부 특강/corpus

In [ ]:
from langchain_community.document_loaders import csv_loader

test = csv_loader.CSVLoader(file_path='./test.csv')
train = csv_loader.CSVLoader(file_path='./train.csv')

doc = test.load() + train.load()

In [ ]:
print(len(doc))

In [ ]:
from konlpy.tag import Okt, Kkma

okt = Okt()
kkma = Kkma()

In [ ]:
def len_okt(text):
  tokens = [token for token in okt.morphs(text)]
  return len(tokens)

def okt_tokenize(text):
  return [token for token in okt.morphs(text)]

In [ ]:
def len_kkma(text):
  tokens = [token for token in kkma.morphs(text)]
  return len(tokens)

def kkma_tokenize(text):
  return [token for token in kkma.morphs(text)]

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n', '\n', ' '],   # 1000토큰을 넘지않고 엔터를 기준으로 split
    chunk_size=1000,
    chunk_overlap=50,
    length_function=len_okt
)

In [ ]:
texts = text_splitter.split_documents(doc)

In [ ]:
texts[0]

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
# model_kwargs = {'device': 'cpu'}  # cpu
model_kwargs = {'device': 'cuda'}  # gpu
encode_kwargs = {'normalize_embeddings': True}
hf_embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs

)

In [ ]:
import getpass, os

if not userdata.get('OPENAI_API_KEY'):
  os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter API Key for OpenAI: ')

from langchain.embeddings import OpenAIEmbeddings

openai_embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [ ]:
# cd /content/drive/MyDrive/디스부 특강

In [ ]:
save_directory = './chroma_docs_db'

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
# !pip install faiss-cpu

In [ ]:
vectorstore = FAISS.from_documents(documents=texts, embedding=embeddings)

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """너는 손자같이 말하는 인공지능이다.
공손하고 따뜻한 말투이어야 한다.
사용자는 할머니라는 사실을 꼭 적용해야 한다.
대화 흐름에 맞는 대답하기 쉬운 질문 하나만 생성한다.

질문은 반드시 한국어로 한 문장만 생성하고,
너무 길거나 복잡하지 않게 해야 한다.


#Previous Chat History:
{chat_history}

#Question:
{question}

#Context:
{context}

#Answer:"""
)


In [ ]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 단계 8: 체인(Chain) 생성
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "chat_history": itemgetter("chat_history"),
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
# 세션 기록을 저장할 딕셔너리
store = {}


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 대화를 기록하는 RAG 체인 생성
rag_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = ChatOpenAI(
    model = 'gpt-4o-mini',
    temperature=0.3,
    streaming=True,   # 한 글자씩 보여줌
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [ ]:
def format_docs(doc):
  # test = "\n\n".join(document.page_content for document in docs)
  # print(test)
  return "\n\n".join(document.page_content for document in doc)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

rag_with_history.invoke(
    # 질문 입력
    {"question": "내가 오늘 뭐 했다고 했지?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "abc123"}},
)